In [53]:
import numpy as np
import torch

In [54]:
# we will make a transforemer based model on shakespear 
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-05-07 15:17:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-05-07 15:17:05 (19.9 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [55]:
!head input.txt

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:


In [56]:
with open("input.txt","r",encoding="utf-8") as f:
    text = f.read()
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [57]:
print(f"length of dataset in characters : {len(text):_}")

length of dataset in characters : 1_115_394


In [58]:
# vocab size
vocab = sorted( list(set(text)))
print(vocab,sep=",")

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [59]:
vocab_size = len(vocab)
vocab_size

65

In [60]:
# since we never terminate this .. we dont need special tokens
stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for s,i in stoi.items()}

encode = lambda word : [stoi[c] for c in word]
decode = lambda code : "".join(itos[s] for s in code)

In [61]:
print(encode("Heylo"))
print(decode(encode("Heylo")))

[20, 43, 63, 50, 53]
Heylo


In [62]:
data = torch.tensor(encode(text))
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [63]:
# lets split the data into train , valid and test split
n1 = int (0.8 * len(data))
n2 = int (0.9 * len(data))
# obviously dont shuffle the data.
data_tr = data[:n1]
data_val = data[n1:n2]
data_test = data[n2:]
print(data_tr.shape , data_val.shape, data_test.shape)

torch.Size([892315]) torch.Size([111539]) torch.Size([111540])


In [64]:
block_size = 8
X_ex = data_tr[:block_size]
print(f"{X_ex=}")
for i,c in enumerate(X_ex):
    print(X_ex[:i],"->",X_ex[i])
    

X_ex=tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([], dtype=torch.int64) -> tensor(18)
tensor([18]) -> tensor(47)
tensor([18, 47]) -> tensor(56)
tensor([18, 47, 56]) -> tensor(57)
tensor([18, 47, 56, 57]) -> tensor(58)
tensor([18, 47, 56, 57, 58]) -> tensor(1)
tensor([18, 47, 56, 57, 58,  1]) -> tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) -> tensor(47)


In [65]:
torch.manual_seed(42)
batch_size = 4 

def mini_batch(split):
    data = data_tr if split == "train" else data_val
    # jump to any point in mini shakespeare 
    ix = torch.randint(0,data.shape[0], (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix],dim=0) # we want examples to be stacked vertically
    y = torch.stack([data[i+1:i+block_size+1] for i in ix],dim=0) # we want examples to be stacked vertically
    return  x,y 

Xb,yb = mini_batch("train")
Xb,yb

(tensor([[61, 47, 58, 46,  1, 57, 50, 53],
         [52,  1, 40, 39, 41, 49,  8,  0],
         [ 0, 21, 58,  1, 58, 46, 43, 52],
         [53, 59, 56,  1, 60, 43, 56, 57]]),
 tensor([[47, 58, 46,  1, 57, 50, 53, 61],
         [ 1, 40, 39, 41, 49,  8,  0,  0],
         [21, 58,  1, 58, 46, 43, 52,  1],
         [59, 56,  1, 60, 43, 56, 57, 43]]))

In [68]:
for batch in range(batch_size):
    for t in range(block_size):
       context = Xb[batch, :t]
       target = Xb[batch, t]
       print(context,"->",target)

tensor([], dtype=torch.int64) -> tensor(61)
tensor([61]) -> tensor(47)
tensor([61, 47]) -> tensor(58)
tensor([61, 47, 58]) -> tensor(46)
tensor([61, 47, 58, 46]) -> tensor(1)
tensor([61, 47, 58, 46,  1]) -> tensor(57)
tensor([61, 47, 58, 46,  1, 57]) -> tensor(50)
tensor([61, 47, 58, 46,  1, 57, 50]) -> tensor(53)
tensor([], dtype=torch.int64) -> tensor(52)
tensor([52]) -> tensor(1)
tensor([52,  1]) -> tensor(40)
tensor([52,  1, 40]) -> tensor(39)
tensor([52,  1, 40, 39]) -> tensor(41)
tensor([52,  1, 40, 39, 41]) -> tensor(49)
tensor([52,  1, 40, 39, 41, 49]) -> tensor(8)
tensor([52,  1, 40, 39, 41, 49,  8]) -> tensor(0)
tensor([], dtype=torch.int64) -> tensor(0)
tensor([0]) -> tensor(21)
tensor([ 0, 21]) -> tensor(58)
tensor([ 0, 21, 58]) -> tensor(1)
tensor([ 0, 21, 58,  1]) -> tensor(58)
tensor([ 0, 21, 58,  1, 58]) -> tensor(46)
tensor([ 0, 21, 58,  1, 58, 46]) -> tensor(43)
tensor([ 0, 21, 58,  1, 58, 46, 43]) -> tensor(52)
tensor([], dtype=torch.int64) -> tensor(53)
tensor([53])

In [ ]:
def compile_dataset(data):
    context_window = torch.z
    

In [75]:
import torch.nn as nn

class BiGram(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        # allowing each cat to have one hot representation for the worst case
        self.embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,x , y):
        logits = self.embedding_table(x)
        # logits are batch_examples , tokens
        # but now tokens after embedding table are vocab_size
        # so logits are batch, (context , vocab_size)
        B, T , R = logits.shape
        # pytorch wants B*T to be the first dimension and R to be the second
        logits = logits.view(B*T,R)
        
        # loss = nn.functional.cross_entropy(x,y)
        return logits
    
    
biGram = BiGram(vocab_size)
out = biGram(Xb,yb) # __call__ -> forward
out.shape , out[0]

(torch.Size([32, 65]),
 tensor([ 0.6733, -0.2337, -0.5862, -0.4561,  0.9994, -1.1085,  1.7156, -0.8135,
          0.0650,  0.0469,  0.9859, -0.7959, -0.2922, -0.8291, -0.5552,  0.7637,
          0.6981, -2.2819, -0.2452, -3.4533, -3.0722,  0.1733, -0.0176, -0.4889,
          2.1990, -0.2319,  0.1248,  0.0940,  0.8477, -1.4172,  0.2188,  2.2037,
          1.4194,  0.2399,  1.1591, -0.2237,  0.5132,  0.6383, -0.1745,  0.5610,
          0.6953,  0.6277,  0.1643, -0.7114,  1.8253,  0.8615,  0.4973,  0.3916,
         -1.2814,  0.4732, -2.2927, -0.4348,  0.7450, -1.2404, -0.7729, -1.5647,
         -0.3501,  1.5520, -1.7492,  0.6099,  1.2775,  0.5749,  1.4199,  0.3876,
          1.9517], grad_fn=<SelectBackward0>))